In [1]:
from dotenv import load_dotenv
load_dotenv()

True

# 데이터 전처리

## 데이터 로드

In [2]:
from langchain.document_loaders import PDFPlumberLoader, TextLoader

import glob

# data 폴더 안의 모든 PDF 가져오기
docs = glob.glob("docling_outputs/*.md")

# docs = ["data/LangGraph-Based Integrated Architecture for CrewAI Multi-Agent.pdf",
#        "data/insight_Agentic.pdf"]

all_docs = []
for d in docs:
    loader = TextLoader(d, encoding="utf-8")
    docs = loader.load()
    all_docs.extend(docs)
    


In [3]:
print(all_docs)

[Document(metadata={'source': 'docling_outputs\\insight_Agentic.pdf.md'}, page_content='## Document 1\n\nJournal of the Korea Institute of Information and Communication Engineering\n\n한국정보통신학회논문지 Vol. 23, No. 1: 399~406, Mar. 2019\n\n## Agentic RAG 시스템 구현 경험에 기반한 설계 인사이트\n\n김재호 1 · 김장영 2*\n\n## Design Insights from Empirical Experiences in Implementing Agentic RAG Systems\n\n## Jae-Ho Kim 1 · Jang-Young Kim 2*\n\n1 Graduate Student, Department of Computer Science, The University of Suwon, Hwaseong, 18323 Korea 2 * Associate Professor, Department of Computer Science, The University of Suwon, Hwaseong, 18323 Korea\n\n## 요 약\n\nRetrieval-Augmented Generation(RAG)과 Fine-tuning 기반 Slim Language Model(SLM)의 구조적 차이 와장단점,그리고AgenticRAG 시스템 설계시고려해야 할핵심요소를분석하였다.텍스트분할방법,임베딩 모델선정, LLM 및 에이전트 워크플로우의 아키텍처 선택이 전체 시스템 성능, 검색 정확도, 안정성에 미치는 영향을살펴보았다. 또한, 무한루프, 분기 조건의 모호성, 도구 선택의 불일치 등 실제로 발생할 수 있는 문제점 과그해결방안도제시하였다.연구결과,자동오류감지및회복로직의구현,도메인특화워크플로우최적화, LLM과 임베딩 모델의 정량적 평가가 중요함을 확인하였다. 더불어 실시간 사용자 피드백 반영이 시스템

In [4]:
len(all_docs)

2

## 데이터 분할(Chunking)

### Semantic Chunking

In [ ]:
# from langchain_experimental.text_splitter import SemanticChunker
# from langchain_ollama import OllamaEmbeddings

# bge_m3_embeddings = OllamaEmbeddings(model="bge-m3")
# semantic_chunker = SemanticChunker(bge_m3_embeddings)

# semantic_chunker_docs = semantic_chunker.split_documents(all_docs)
# len(semantic_chunker_docs)

21

### Recursive Character Text Splitter

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 일반 텍스트용 splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=100
)
split_documents = text_splitter.split_documents(all_docs)

In [14]:
len(split_documents)

86

# SAVE: Qdant Vector Store

In [ ]:
from langchain_ollama import OllamaEmbeddings
from langchain_qdrant import QdrantVectorStore, FastEmbedSparse, RetrievalMode
from qdrant_client import QdrantClient, models
from qdrant_client.http.models import Distance, VectorParams, SparseVectorParams

In [24]:
# client = QdrantClient(":memory:") → 인메모리 벡터 저장소 생성.
client = QdrantClient(host="localhost", port=6333) # docker를 이용해 사용. 따라서 포트번호가 필요함.

collection_name = "RAG_Template"

dense_embedding = OllamaEmbeddings(model="bge-m3")
sparse_embedding = FastEmbedSparse(model_name="Qdrant/bm25")

In [27]:
# Collection 생성
client.create_collection(
    collection_name=collection_name,
    vectors_config={"dense": VectorParams(size=1024, distance=Distance.COSINE)},
    sparse_vectors_config={"sparse": SparseVectorParams(index=models.SparseIndexParams(on_disk=False))},
)

True

In [28]:
qdrant = QdrantVectorStore(
    client=client,
    collection_name=collection_name,
    embedding=dense_embedding,
    sparse_embedding=sparse_embedding,
    retrieval_mode=RetrievalMode.HYBRID,
    vector_name="dense",
    sparse_vector_name="sparse"
)

qdrant.add_documents(split_documents)

['f68e452f0152426c8014cecaf5ef32dd',
 'fa8a2cfee5954cdba29eff36201045d4',
 '3d900874c65a46e48a1f476191d646cb',
 '27c523eeb47d44609154fe25d01ea2f4',
 'f2ef71f6e19f4dfeb9b123b9882cbf29',
 'ddb2880859d646168e6b374a2a949ad2',
 '01a8961c635a4adebb5fe443811f0df6',
 'e8eb10ebae8c45f18a2123600e8f1424',
 'ab4deb4a72bd416fab60f1707273d33a',
 'eff19cee6b6b4df197ca799914fa2c5d',
 '24fd77272d33440086c605d7e84a5aaa',
 '22e7caefd82340f58c01a7159915e247',
 'abb52489ec2f46e5b7c992a5115dbbb8',
 '144dc4407b574005b562fe1349c030af',
 'b9b77e8ce2d4473bac51fc592dfc09b0',
 '665201220de34df1b10a9e7c4ee2ff8b',
 'cc239b7d9ed14cb69f808c3b8bd18cb4',
 'ef713baac9194439abf652e18204f1cb',
 '0b97e1fa643e4713aa1b3decc7c525f8',
 '03c8d941491544c2bfb8b0336e00cf5f',
 'a20397af7f3448bab356e29134f0c0b7',
 'ddc9498ac9744efd946d836c5daf7de4',
 '743144d9dd574734b3426b85896006f9',
 '324182df84cc47cea826b17696aea0a4',
 '6711799458a240b78638e1047126a764',
 'b4fe0d1486d244d8a3b5408761602c19',
 'e1ac8710b490438eb5db8cc82525fddd',
 

In [29]:
response = qdrant.as_retriever(search_kwargs={"k": 4})

In [30]:
query = "self-rag"
print(response.invoke(query)[0].page_content)

그림 2는 법률 도메인에서 세 가지 RAG 구성 (Advanced RAG, Craw AI 단일 Retriever, Craw AI Multi Tool)의 네 가지 평가 지표(정확성, 완전성, 명 료성, 종합점수)별 성능을 시각적으로 비교한 것이다. Advanced RAG는 대부분의 항목에서 고르게 높은 점수(9점대)를 기록했으나, Metric 4에서는 7.4점으로 다소 부족함을 보였다. Craw AI (Only Retriever)는 전반적으로 낮은 점수(78점대)를 기록하여, 단일 검 색만으로는 법률 문서의 정확성과 완전한 커버리지 에 한계가 있음을 시사한다. 반면 Craw AI (Multi Tool)는 모든 지표에서 가장 높은 점수(69.13)를 보 여, 툴 결합이 Retrieval-Generation 품질을 현저히 향상시킴을 실증적으로 보여준다.


In [32]:
response.invoke(query)

[Document(metadata={'source': 'docling_outputs\\LangGraph-Based Integrated Architecture for CrewAI Multi-Agent.pdf.md', '_id': '8b3a9c1d-113f-4bd9-8060-409a9e421cf7', '_collection_name': 'RAG_Template'}, page_content='그림 2는 법률 도메인에서 세 가지 RAG 구성 (Advanced RAG, Craw AI 단일 Retriever, Craw AI Multi Tool)의 네 가지 평가 지표(정확성, 완전성, 명 료성, 종합점수)별 성능을 시각적으로 비교한 것이다. Advanced RAG는 대부분의 항목에서 고르게 높은 점수(9점대)를 기록했으나, Metric 4에서는 7.4점으로 다소 부족함을 보였다. Craw AI (Only Retriever)는 전반적으로 낮은 점수(78점대)를 기록하여, 단일 검 색만으로는 법률 문서의 정확성과 완전한 커버리지 에 한계가 있음을 시사한다. 반면 Craw AI (Multi Tool)는 모든 지표에서 가장 높은 점수(69.13)를 보 여, 툴 결합이 Retrieval-Generation 품질을 현저히 향상시킴을 실증적으로 보여준다.'),
 Document(metadata={'source': 'docling_outputs\\insight_Agentic.pdf.md', '_id': '4ae219f2-9ebf-4e52-9e78-052532327da1', '_collection_name': 'RAG_Template'}, page_content='## Ⅴ. 결론 및 향후연구\n\n본 논문에서는 Fine-tuning 기반 SLM과 RAG 의 구조적 차이와 장단점을 비교 분석하고, RAG 와 Agentic RAG 시스템 설계 시 고려해야 할 주요 이슈들을 실험적으로 제시하였다. "다양한 실험과 분석을 통해 텍스트 분할 방식, 임베딩'),
 Document(metadata={